# Praktikum 5 - Sentiment Analysis
Sentiment Analysis merupakan salah satu bahasan dalam Natural Language Processing (NLP). Sentiment Analysis adalah proses untuk mendefinisikan sebuah teks termasuk ke dalam kategori <b>positif, negatif, </b>atau <b>neutral</b>. Hal ini juga dikenal sebagai opinion mining, berdasarkan opini atau sikap dari penulis/pembicara.

## Agenda
1. Simple Twitter Sentiment Analysis using NLTK
2. Airline Twitter Sentiment Analysis (14640 tweets) and Predict New Tweet
3. Sentiment Analysis Review Movies using Gensim Doc2Vec

## 1. Simple Twitter Sentiment Analysis using NLTK
Pada bagian ini, kita menggunakan 15 sample tweets yang 5 tweets berlabel positif, 5 tweets berlabel negatif, dan 5 tweets untuk test. Sentiment Analysis ini berdasarkan tutorial yang terdapat di [sini](http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/). 

In [ ]:
pos_tweets = [('I love this car', 'positive'),
              ('This view is amazing', 'positive'),
              ('I feel great this morning', 'positive'),
              ('I am so excited about the concert', 'positive'),
              ('He is my best friend', 'positive')]

neg_tweets = [('I do not like this car', 'negative'),
              ('This view is horrible', 'negative'),
              ('I feel tired this morning', 'negative'),
              ('I am not looking forward to the concert', 'negative'),
              ('He is my enemy', 'negative')]

test = [('I feel happy this morning', 'positive'),
        ('Larry is my friend', 'positive'),
        ('I do not like that man', 'negative'),
        ('My house is not great', 'negative'),
        ('Your song is annoying', 'negative')]

### Preprocessing
- Dengan preprocessing sederhana, yaitu hanya menggunakan kata yang jumlah karakter lebih dari 3 akan termasuk data yang akan diproses.

In [ ]:
tweets = []
for (words, sentiment) in pos_tweets + neg_tweets:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

- Hasil dari list tweets yang menggabungkan positif dan negatif tweets.

In [ ]:
print(tweets)

- Lakukan hal yang sama pada test tweets

In [ ]:
test_tweets = []
for (words, sentiment) in test:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    test_tweets.append((words_filtered, sentiment))

- Hasil list dari test tweets

In [ ]:
print(test_tweets)

### Classifier
Classifier yang digunakan adalah naive bayes.
- List dari fitur kata pada variable <b>tweets</b> dilakukan ekstraksi. Sehingga diketahui jumlah frekuensi kemunculan kata

In [ ]:
import nltk

def get_words_in_tweets(tweets):
    all_words = []
    for(words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))

In [ ]:
print(word_features)

- Untuk membuat sebuah classifier, kita butuh untuk menentukan fitur apa saja yang relevan. Sehingga kita butuh feature extractor.

In [ ]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

- Sebagai contoh, terdapat document dengan kata <b>'love', 'this', 'car'</b> kemudian dilakukan extraksi fitur kata.

In [ ]:
doc = ['love', 'this', 'car']

print(extract_features(doc))

- Dengan adanya feature extractor, kita bisa menerapkan fitur pada classifier menggunakan fungsi <b>apply_features</b>.

In [ ]:
training_set = nltk.classify.apply_features(extract_features, tweets)

In [ ]:
print(training_set)

- Setelah berhasil memiliki training set, sekarang kita dapat melakukan training pada classifier. Classifier yang digunakan adalah Naive Bayes

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

Naive Bayes Classifier menggunakan prior probability pada setiap label yang merupakan frekuensi tiap label pada training set, dan kontribusi dari fitur.

### Classify
Setelah kita memiliki classifier, kita bisa melakukan klasifikasi sebuah tweet dan melihat termasuk kategori apa tweet tersebut. Berikut adalah contohnya.

In [ ]:
tweet = 'Larry is my friend'
print(classifier.classify(extract_features(tweet.split())))

Tweet di atas termasuk kategori <b>positif</b> karena kata <b>friend</b> berasosiasi pada tweet positif <b>He is my best friend</b>

In [ ]:
# mengeluarkan hasil positive karena feature name 'annoying' 
# belum terdapat pada data training
tweet2 = 'Your song is annoying'
print(classifier.classify(extract_features(tweet2.split())))

Sedangkan pada tweet di atas, kata <b>annoying</b> belum terdaftar sebagai salah satu fitur di training set.

## 2. Airline Twitter Sentiment Analysis (14640 tweets) and Predict New Tweet
Pada bagian kedua, terinspirasi dari [kaggle](https://www.kaggle.com/) dalam melakukan sentiment analysis tweet airline. 

### Import Files dan Packages
Sebagai persiapan download dataset <b>Tweets.csv</b> yang berisi tweet airline pada link [berikut](https://www.kaggle.com/crowdflower/twitter-airline-sentiment). Pastikan letakkan file pada lokasi yang berdekatan dengan file .ipynb. Pada praktikum ini, kami letakkan satu lokasi dengan file .ipynb 

In [ ]:
# Import File and Packages
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
import re
import warnings
warnings.filterwarnings("ignore")

# Read csv file into data frame
tweet=pd.read_csv("Tweets.csv")

### Load and preprocess data using NLTK
<b><i>Tokenize, Clean, Stem, Lemmatize, Remove stopwords</i></b>

In [ ]:
# Preprocess the data {'negative': 0 , 
# 'positive': 1 , 'neutral': 2}

df = tweet.iloc[:,(10,1)]
df.columns = ['data', 'target']
df['target'] = df['target'].str.strip().str.lower()
df['target'] = df['target'].map({'negative': 0 , 'positive': 1 , 'neutral': 2})

# Copy df to a temporary dataframe for pre-processing
dft = df

In [ ]:
%%time
# Remove @tweets, numbers, hyperlinks that do not start with letters
dft['data'] = dft['data'].str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([0-9])"," ")

In [ ]:
%%time
# tokenize into words
import nltk
dft['data'] = dft['data'].apply(nltk.word_tokenize)

In [ ]:
%%time
# stem the tokens
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
dft['data'] = dft['data'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
%%time
# Lemmatizing
lemmatizer = nltk.WordNetLemmatizer()
dft['data'] = dft['data'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [ ]:
%%time
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')

# stem the stopwords
stemmed_stops = [stemmer.stem(t) for t in stopwords]

# remove stopwords from stemmed/lemmatized tokens
dft['data'] = dft['data'].apply(lambda x: [stemmer.stem(y) for y in x if y not in stemmed_stops])

# remove words whose length is <3
dft['data'] = dft['data'].apply(lambda x: [e for e in x if len(e) >= 3])

In [ ]:
%%time
# Detokenize cleaned dataframe for vectorizing
dft['data'] = dft['data'].str.join(" ") 

### Printing data shape:
mencetak shape dari dataset yang akan diproses.

In [ ]:
# Print attributes of tweet, X and y
print('Shape of original file : ', tweet.shape)
print('All columns of the original file : ', tweet.columns.tolist() , '\n')
print('Columns dft dataframe : ',dft.columns.tolist(), '\n') 
print('Shape data and target : ', dft['data'].shape, dft['target'].shape, '\n')
print('Mood Count target :\n', tweet['airline_sentiment'].value_counts())

### Initialize X y :

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

X = dft['data']
y = dft['target']

### Print top features with freaquency:

In [ ]:
def print_top_words():    
    # Print top words
    vect = CountVectorizer(stop_words='english',analyzer="word", min_df = 2, max_df = 0.8)
    data_dtm = vect.fit_transform(dft['data'])
    feat_dtm = vect.get_feature_names()

    # Count words
    freq_tbl = pd.DataFrame({'Word':feat_dtm,'Occurence':np.asarray(data_dtm.sum(axis=0)).ravel().tolist()})
    freq_tbl['Word'] = freq_tbl['Word'].str.strip()

    # Print top words
    topt = freq_tbl.sort(['Occurence'], ascending=[False]).head(10)
    y = topt['Occurence']
    plt.grid()
    X = range(1, 11)
    plt.bar(X,y,color='g')
    plt.xlabel('Top words')
    plt.ylabel('Occurence')
    plt.title('Frequency of top 10 words')
    plt.xticks(X,topt['Word'],rotation=90)
    
def print_top_neg_words():    
    # Print top negative words
    vect = CountVectorizer(stop_words='english',analyzer="word", min_df = 2, max_df = 0.8)
    filt = dft[dft['target'] == 0]
    data_dtm = vect.fit_transform(filt['data'])
    feat_dtm = vect.get_feature_names()

    # Count words
    freq_tbl = pd.DataFrame({'Word':feat_dtm,'Occurence':np.asarray(data_dtm.sum(axis=0)).ravel().tolist()})
    freq_tbl['Word']=freq_tbl['Word'].str.strip()

    # Print top negative words
    topt = freq_tbl.sort(['Occurence'], ascending=[False]).head(10)
    y = topt['Occurence']
    plt.grid()
    X = range(1, 11)
    plt.bar(X,y,color='g')
    plt.xlabel('Top negative words')
    plt.ylabel('Occurence')
    plt.title('Frequency of top 10 negative words')
    plt.xticks(X,topt['Word'],rotation=90)
    
def print_top_pos_words():    
    # Print top positive words
    vect = CountVectorizer(stop_words='english',analyzer="word", min_df = 2, max_df = 0.8)
    filt = dft[dft['target'] == 1]
    data_dtm = vect.fit_transform(filt['data'])
    feat_dtm = vect.get_feature_names()

    # Count words
    freq_tbl = pd.DataFrame({'Word':feat_dtm,'Occurence':np.asarray(data_dtm.sum(axis=0)).ravel().tolist()})
    freq_tbl['Word']=freq_tbl['Word'].str.strip()

    # Print top positive words
    topt = freq_tbl.sort(['Occurence'], ascending=[False]).head(10)
    y = topt['Occurence']
    plt.grid()
    X = range(1, 11)
    plt.bar(X,y,color='g')
    plt.xlabel('Top positive words')
    plt.ylabel('Occurence')
    plt.title('Frequency of top 10 positive words')
    plt.xticks(X,topt['Word'],rotation=90)

### Draw in matplotlib:

In [ ]:
plt.figure(1,figsize=(16, 16))
plt.subplot(251)
print_top_words()  
plt.subplot(253)
print_top_pos_words()
plt.subplot(255)
print_top_neg_words()

### Classifying new tweets: Fit model, Clean Tweet, Predict Mood

#### Fit Model : Logistic Regression 

In [ ]:
%%time
import time
# Train Test split data with random state = 11
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, random_state=11)

# Vectorize
vect = CountVectorizer(stop_words='english',analyzer="word",min_df = 2, max_df = 0.8)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
feat_dtm = vect.get_feature_names()

# Initialize classifier stats
clf_stats = pd.DataFrame()

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

start_time = time.time()
clf.fit(X_train_dtm, y_train)
runtime = time.time()-start_time
y_pred = clf.predict(X_test_dtm)
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy : ',accuracy)

# Store stats for classifier
clf_stats = clf_stats.append({'Classifier': 'Logistic Regression',
                              'Accuracy': accuracy, 'Runtime': runtime, 'Callable': 'clf = LogisticRegression()'}, 
                             ignore_index=True)

#### Clean incoming new tweet:

In [ ]:
# Clean input tweet

def fmt_input_tweet(txt):
    
    # Remove @tweets, numbers, hyperlinks that do not start with letters
    txt = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([0-9])"," ",txt)
    #print(txt)
    
    # tokenize into words
    tokens = [word for word in nltk.word_tokenize(txt)]
    #print(tokens)

    # only keep tokens that start with a letter (using regular expressions)
    clean_tokens = [token for token in tokens if re.search(r'^[a-zA-Z]+', token)]
    #print('clean_tokens:\n',clean_tokens)

    # stem the tokens
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(t) for t in clean_tokens]
    #print('stemmed_tokens:\n',stemmed_tokens)

    #Lemmatizing
    lemmatizer = nltk.WordNetLemmatizer()
    lem_tokens = [lemmatizer.lemmatize(t) for t in stemmed_tokens]
    #print('lemmatizer : \n',lem_tokens)
    
    #Remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')

    # stem the stopwords
    stemmed_stops = [stemmer.stem(t) for t in stopwords]

    # remove stopwords from stemmed/lemmatized tokens
    lem_tokens_no_stop = [stemmer.stem(t) for t in lem_tokens if t not in stemmed_stops]

    # remove words whose length is <3
    clean_lem_tok = [e for e in lem_tokens_no_stop if len(e) >= 3]
    #print('clean_lem_tok: ',clean_lem_tok)
    
    # Detokenize new tweet for vector processing
    new_formatted_tweet=" ".join(clean_lem_tok)
    #print('new_formatted_tweet: ',new_formatted_tweet)
    
    return new_formatted_tweet

### Classify incoming new tweet:

In [ ]:
# Vectorize, fit, transform. Select model randomly
vect = CountVectorizer(stop_words='english', analyzer="word", min_df = 2, max_df = 0.8)
X_dtm = vect.fit_transform(X)
feat_dtm = vect.get_feature_names()

# Select the best performing classifier
Call_clf = str(clf_stats[['Callable','Accuracy']].sort(['Accuracy'], ascending=[False]).head(1).iloc[:,(0)])
temp = Call_clf.__repr__()
Call_clf = temp[temp.index('c'):(temp.index(')'))+1]
print('Model :',temp[(temp.index('=') + 1) : temp.index('(')])
exec(Call_clf)
clf.fit(X_dtm.toarray(), y) 

def classify_new_tweet(new_twt):  

    fmt_twt = fmt_input_tweet(new_twt)
    fmt_twt_dtm = vect.transform([fmt_twt])[0]
    #print('Formatted Tweet :',fmt_twt)
    pred = clf.predict(fmt_twt_dtm.toarray())

    def mood(x):
        return {
            0: 'negative',
            1: 'positive',
            2: 'neutral'
        }[x]

    print('Mood of the incoming tweet is:',mood(pred[0]))

### Predict mood new tweet:

In [ ]:
# New input tweet
twt='@united I am sick!! https://www.abc.com'
classify_new_tweet(twt)

## 3. Sentiment Analysis Review Movies using Gensim Doc2Vec
Pada bagian ketiga, terinspirasi dari [word2vec-sentimen](http://linanqiu.github.io/2015/10/07/word2vec-sentiment/) dalam melakukan sentiment analysis pada review movie yang sangat mudah dilakukan.

## Sentiment Analysis using Doc2Vec
Menggunakan <b>Word2Vec</b> sangatlah mudah, secara singkat proses yang dilakukan yaitu terdapat input sebuah <b>corpus</b>, yang mengeluarkan hasil berupa <b>vectors dari setiap kata.</b>

Dengan keluaran vectors ini, dapat membuat kata yang serupa akan saling berkaitan/berdekatan. Sebagai contoh kata <b>v_man - v_woman</b> sangat berkaitan dengan <b>v_king - v_queen</b>.
Proses ini, dalam dunia NLP disebut dengan <b>word embedding</b> yaitu nama kolektif dalam language modelling dimana kata-kata atau frase dari kosakata dipetakan ke dalam vector.

Dengan menggunakan Doc2Vec, kita bisa mewakilkan sebuah kalimat dengan sebuah vector dan dengan mudah kita dapat menjalankan algoritma klasifikasi sesuka kita. Bukankah menarik?    

## Setup
## Modules
Disini menggunakan <b>gensim</b>, karena <b>gensim</b> memiliki banyak implementasi pada Word2Vec (dan Doc2Vec). Dan juga menggunakan <b>numpy</b> untuk manipulasi array dan <b>sklearn</b> untuk classifier Logistic Regression.

In [ ]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# random
from random import shuffle

# classifier
from sklearn.linear_model import LogisticRegression

## Input Format
Data input yang digunakan diambil dari Cornell IMDB movie review corpus (http://www.cs.cornell.edu/people/pabo/movie-review-data/). Namun data tersebut masih berupa raw.

Oleh karena itu, dapat menggunakan data yang sudah dilakukan <b>clean</b> dengan dilakukan convert ke dalam lower case dan menghapus punctuation. Untuk data yang telah diolah dapat diperoleh di [sini](https://github.com/yasirabd/Prak-TKH/tree/master/Prak-5). Letakkan files pada satu lokasi dengan .ipynb.

Berikut ini adalah document tersebut:
- test-neg.txt: 12500 negative movie reviews from the test data
- test-pos.txt: 12500 positive movie reviews from the test data
- train-neg.txt: 12500 negative movie reviews from the training data
- train-pos.txt: 12500 positive movie reviews from the training data
- train-unsup.txt: 50000 Unlabelled movie reviews

Sebagai catatan untuk mengecek document, <b>Setiap document terdapat dalam satu baris, dan dipisahkan dengan baris baru ke document berikutnya.</b>

## Feeding Data to Doc2Vec
Fungsi Doc2Vec pada <b>gensim</b> memiliki kelemahan pada membaca data pada sebuah files. Hal ini dikarenakan pada class <b>LabeledLineSentence</b> yang berasal dari <b>LabeledSentence</b> (sebuah class dari <b>gensim.models.doc2vec</b>) hanya mewakilkan dari <b>satu kalimat</b>.

Perbedaan dari Word2Vec dan Doc2Vec sendiri adalah

Word2Vec melakukan convert sebuah kata menjadi vector.

Doc2Vector melakukan convert seluruh kata pada kalimat menjadi sebuah vector. Oleh karena itu butuh <b>kata yang spesial</b> sebagai label pada kalimat tersebut.

Sehingga format yang diinginkan adalah sebagai berikut:<br>
<b>[['word1', 'word2', 'word3', 'lastword'], ['label1']]</b>

<b>LabeledSentece</b> sudah sangat lebih rapi dalam membuat format seperti di atas yaitu menjadi - sebuah list dari kata dan label.

Masalahnya pada kelas <b>LabeledLineSentence</b> hanya bisa melakukan "perubahan" pada satu files saja. Sedangkan kita butuh untuk multiple files, seperti misalnya pada multi documents (test, training, positive, negative, etc).

Oleh karena itu kita akan membuat kelas <b>LabeledLineSentence</b> sendiri. Berikut adalah kelas yang telah dimodifikasi:

In [ ]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), 
                                                          [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

Dengan adanya kelas <b>LabeledLineSentence</b> yang baru, kita dapat melakukan input data file lebih dari satu. Kita perlu memasukkan <b>nama file</b> dan <b>special prefixes yang unik</b> pada tiap document.

Berikut ini adalah penerapan pada praktikum:

In [ ]:
sources = {'test-neg.txt':'TEST_NEG', 
           'test-pos.txt':'TEST_POS', 
           'train-neg.txt':'TRAIN_NEG', 
           'train-pos.txt':'TRAIN_POS', 
           'train-unsup.txt':'TRAIN_UNS'}

sentences = LabeledLineSentence(sources)

## Model
## Building the Vocabulary Table
Doc2Vec membutuhkan sebuah <b>vocabulary table</b> yang akan memproses semua kata dan menyaring kata-kata yang unik serta melakukan perhitungan dasar. <b>model.build_vocab</b> butuh input sebuah array, oleh karena itu kita gunakan fungsi <b>to_array</b> dari kelas <b>LabeledLineSentences</b> yang telah didefinisikan di atas.

Jika ingin mengetahui parameter pada Word2Vec documentation, berikut adalah penjelasannya:
- <b>min_count</b>: ignore all words with total frequency lower than this. You have to set this to 1, since the sentence labels only appear once. Setting it any higher than 1 will miss out on the sentences.
- <b>window</b>: the maximum distance between the current and predicted word within a sentence. Word2Vec uses a skip-gram model, and this is simply the window size of the skip-gram model.
- <b>size</b>: dimensionality of the feature vectors in output. 100 is a good number. If you’re extreme, you can go up to around 400.
- <b>sample</b>: threshold for configuring which higher-frequency words are randomly downsampled
- <b>workers</b>: use this many worker threads to train the model

In [ ]:
%%time
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, 
                workers=8)

model.build_vocab(sentences.to_array())

## Training Doc2Vec
Langkah selanjutnya adalah melakukan training pada model. <b>Model dilakukan training jika setiap epoch saat training, urutan dari kalimat yang dimasukkan dalam model dilakukan random.</b> Inilah alasan menggunakan fungsi <b>sentences_perm</b> pada kelas <b>LabeledLineSenteces</b>.

Langkah ini termasuk yang paling lama. Pada Laptop Core i3 Ram 8GB membutuhkan waktu sekitar <b>1h 56min 39s.</b>

In [ ]:
%%time
for epoch in range(10):
    model.train(sentences.sentences_perm())

## Saving and Loading Models
Untuk mencegah melakukan training data yang butuh waktu lama, kita bisa menyimpannya dengan cara berikut.

In [ ]:
%%time
model.save('./imdb.d2v')

Dan untuk melakukan load dengan cara berikut.

In [ ]:
%%time
model = Doc2Vec.load('./imdb.d2v')

## Inspecting the Model
Sekarang kita lihat model yang dihasilkan. Misalkan kata <b>good</b> kita dapat mencari kata yang paling mirip dengan <b>good</b>. 

In [ ]:
model.most_similar('good')

Kita juga bisa melihat vector hasil pembetukan model tersebut, misalkan kita ingin melihat contoh vector dari kalimat pertama pada training set untuk negative reviews:

In [ ]:
model.docvecs['TRAIN_NEG_0']

## Classifying Sentiments
## Training Vectors
Sekarang kita gunakan vector tersebut untuk melakukan training pada classifier. Pertama, kita harus melakukan extract vector training. Disini kita memiliki 25000 training reviews, dimana positive 12500 dan negative 12500.

Kemudian kita harus membentuk array dengan <b>numpy</b>, dimana akan dibentuk 2 array paralel, yang satu berisi vector <b>(train_arrays)</b> dan yang lainnya berisi label <b>(train_labels)</b>.

Disini kita meletakkan positive reviews pertama kali, kemudian selanjutnya negative reviews.

In [ ]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

Training Array akan tampak seperti berikut: dimana setiap baris mewakilkan setiap kalimat.

In [ ]:
print(train_arrays)

Sedangkan pada label vector yang dihasilkan yaitu, 1 mewakilkan positive sedangkan 0 mewakilkan negative.

In [ ]:
print(train_labels)

## Testing Vectors
Kita akan melakukan hal yang sama pada data testing. Hasil dari data testing dapat kita gunakan untuk melakukan evaluasi dari hasil.

In [ ]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[12500 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

## Classification
Selanjutnya kita lakukan training untuk <b>Logistic Regression</b> classifier menggunakan data training.

In [ ]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

Berikut adalah hasi accuracy yang didapatkan.

In [ ]:
classifier.score(test_arrays, test_labels)